In [ ]:
import re
import pandas as pd
from selenium import webdriver

In [ ]:
# Credentials
USERNAME = ''
PASSWORD = ''

# Bool for scraping FR descriptions
FRENCH = True

In [ ]:
browser = webdriver.Chrome()

In [ ]:
# Navigate to GCcampus and login
if FRENCH:
    main_url = 'https://idp.csps-efpc.gc.ca/idp/login-fr.jsp'
else:
    main_url = 'https://idp.csps-efpc.gc.ca/idp/Authn/UserPassword'

browser.get(main_url)
browser.find_element_by_id('j_username').send_keys(USERNAME)
browser.find_element_by_id('j_password').send_keys(PASSWORD)
browser.find_element_by_id('cbPrivacy').click()
browser.find_element_by_xpath("//button[@type='submit']").click()

In [ ]:
%%time
# Loop through catalogue and get all links to courses
if FRENCH:
    list_url = 'https://learn-apprendre.csps-efpc.gc.ca/application/fr/courses-solr?page='
else:
    list_url = 'https://learn-apprendre.csps-efpc.gc.ca/application/en/courses-solr?page='

course_links = []
for i in range(43): # Limit of ?page=42 in EN (therefore use range(43)); ?page=42 in FR
    browser.get(list_url + str(i))
    mars = browser.find_elements_by_css_selector('.field-items a')
    for elem in mars:
        course_links.append(elem.get_attribute('href'))

In [ ]:
%%time
# For each link in 'course_links', navigate to page, grab course description (HTML
# tags included), and save to 'desc_dict'
desc_dict = {}
for link in course_links:
    browser.get(link)
    desc = browser.find_elements_by_css_selector('.field-item[property="content:encoded"]')[0].get_attribute('innerHTML')
    desc_dict[link] = desc

In [ ]:
# Change dict keys from links to course codes
desc_dict_processed = {}

regex = re.compile(pattern=r'[a-zA-Z]{1}\d{3}(?:-module-\d{1}|-1|-2)?')
for key, val in desc_dict.items():
    try:
        key_processed = regex.findall(key)[0].upper().replace('-MODULE-', ' - Module ')
    except:
        key_processed = key
    desc_dict_processed[key_processed] = val

In [ ]:
# Export 'desc_dict' to CSV
df = pd.DataFrame.from_dict(desc_dict_processed, orient='index')
df.to_csv('scraped_{0}.csv'.format('fr' if FRENCH else 'en'), sep=',', encoding='utf-8')